In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import os
import time#可以用来简单地记录时间
import matplotlib.pyplot as plt#画图
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import  average_precision_score

import torch#深度学习的pytoch平台
from torch import Tensor
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
from models.itransformer import itransformerModel
from models.lstm_itransformer import lstm_itransformerModel
from models.lstm import LSTMModel
# from trainer.train_and_evaluate_2 import ModelTrainer, ModelEvaluator
# from data_provider.dataset_generate import TimeSeriesDataset

## 加载数据

In [3]:
database_names_all = ['ams', 'eicu', 'mimic', 'salz']

selected_database_num = 2
internal_database = database_names_all[selected_database_num]  # ''
external_database = database_names_all.copy()
external_database.remove(internal_database) # []
print(internal_database, external_database)

mimic ['ams', 'eicu', 'salz']


In [4]:
file_path = "E:\\Research\\Time series research\\Federated learning Time series research\\0.data\\Multi-center time series data\\"
internal_data_path = 'icu_mortality_' + internal_database + '.csv'

In [5]:
df_internal = pd.read_csv(file_path + internal_data_path)
df_internal

,id,gender,age,height,weight,bmi,admission_type,death_hosp,los_icu_day,ethnicity,...,oasis,sapsii,respiration,coagulation,liver,cardiovascular,cns,renal,sofa,mods
0,5,0,1.017520,-0.021828,-0.79298,-0.998913,1,0,1.73,other,...,33.0,35.0,0,1,0,1,4,4,10.0,0
1,5,0,1.017520,-0.021828,-0.79298,-0.998913,1,0,1.73,other,...,33.0,37.0,0,1,0,1,4,4,10.0,0
2,5,0,1.017520,-0.021828,-0.79298,-0.998913,1,0,1.73,other,...,33.0,37.0,0,1,0,1,4,4,10.0,0
3,5,0,1.017520,-0.021828,-0.79298,-0.998913,1,0,1.73,other,...,35.0,37.0,0,1,0,1,4,4,10.0,0
4,5,0,1.017520,-0.021828,-0.79298,-0.998913,1,0,1.73,other,...,33.0,35.0,0,1,0,1,4,4,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918410,76271,1,-0.637632,0.583969,-0.54191,-0.931153,0,0,1.17,white,...,6.0,21.0,2,0,0,1,1,0,4.0,0
3918411,76271,1,-0.637632,0.583969,-0.54191,-0.931153,0,0,1.17,white,...,6.0,21.0,0,0,0,0,0,0,NaN,0
3918412,76271,1,-0.637632,0.583969,-0.54191,-0.931153,0,0,1.17,white,...,6.0,21.0,0,0,0,0,0,0,NaN,0
3918413,76271,1,-0.637632,0.583969,-0.54191,-0.931153,0,0,1.17,white,...,6.0,21.0,0,0,0,0,0,0,NaN,0


In [6]:
print(df_internal.groupby('id')['death_hosp'].last().value_counts())

death_hosp
0    35436
1     5614
Name: count, dtype: int64


In [7]:
train_ids, test_ids = train_test_split(
    df_internal['id'].unique(),  # 按患者ID划分
    test_size=0.2,      # 测试集比例
    random_state=42,    # 随机种子
    stratify=df_internal.groupby('id')['death_hosp'].last()  # 按患者最终标签分层
)
train_ids

array([40804, 52798, 37874, ..., 24565, 47758, 43247], dtype=int64)

In [8]:
# 获取划分后的DataFrame
train_df = df_internal[df_internal['id'].isin(train_ids)]
test_df = df_internal[df_internal['id'].isin(test_ids)]

print("训练集比例:")
print(train_df.groupby('id')['death_hosp'].last().value_counts(normalize=True))
print("\n测试集比例:")
print(test_df.groupby('id')['death_hosp'].last().value_counts(normalize=True))

训练集比例:
death_hosp
0    0.863246
1    0.136754
Name: proportion, dtype: float64

测试集比例:
death_hosp
0    0.863216
1    0.136784
Name: proportion, dtype: float64


## 数据生成器

In [21]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, feature_cols, window_size=24, forecast_horizon=24, stride=1, 
                 mode='sliding', shuffle=True, label=['death_hosp'], random_state=42, max_len=None):
        """
        初始化时间序列数据集
        
        参数:
            df: 包含所有数据的DataFrame
            feature_cols: 使用的特征列名列表
            window_size: 输入序列长度
            forecast_horizon: 预测时间范围
            mode: 'sliding'滑动窗口，'cumulative'累积窗口，‘fix’固定时间窗口
            shuffle: 是否打乱数据顺序
            random_state: 随机种子
            label: 标签
        """
        self.df = df
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mode = mode
        self.shuffle = shuffle
        self.random_state = random_state
        self.max_len = max_len
        self.indices = []
        self.stride = stride
        self.label = label
        

        # if len(label)==1:
        #     self.label = label[0]
        # else:
        #     self.label = label
        
        # 预计算所有可能的序列索引
        self._precompute_indices()
        
    def _precompute_indices(self):
        """计算所有有效的序列索引"""
        random.seed(self.random_state)
        
        for pid, group in tqdm(self.df.groupby('id')):
            group = group.sort_values('hr')
            max_hr = group['hr'].max()
            
            if self.mode == 'sliding':
                for start in range(1, max_hr - self.window_size, self.stride):
                    end = start + self.window_size-1
                    forecast_end = end + self.forecast_horizon
                    if len(group[(group['hr'] >= start) & (group['hr'] <= end)]) == self.window_size:
                        y = []
                        for label in self.label:
                            condition = (group['hr'] >= end) & (group['hr'] <= forecast_end) & (group[label] == 1)
                            y.append(int(condition.any()))
                        self.indices.append((pid, start, end, y))
                        
            elif self.mode == 'cumulative':
                # for end in range(max_hr, max_hr+1, self.stride):
                # for end in range(13, 14, self.stride):
                for end in range(1, max_hr + 1, self.stride):
                    start = 1
                    forecast_end = end + self.forecast_horizon
                    y = []
                    for label in self.label:
                        condition = (group['hr'] >= end) & (group['hr'] <= forecast_end) & (group[label] == 1)
                        y.append(int(condition.any()))
                    self.indices.append((pid, start, end, y))
                    
            elif self.mode == 'fix':
                start = 1
                end = start + self.window_size
                if len(group[(group['hr'] >= start) & (group['hr'] < end)]) == self.window_size:
                    y = []
                    for label in self.label:
                        condition = (group['hr'] >= end) & (group[label] == 1)
                        y.append(int(condition.any()))
                    self.indices.append((pid, start, end, y))
                        
                    # # condition = (group['hr'] >= end) & (group['death_hosp'] == 1)
                    # condition = group['death_hosp'] == 1
                    # y = int(condition.any())
                    # self.indices.append((pid, start, end, y))
                
        
        if self.shuffle:
            random.shuffle(self.indices)
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        pid, start, end, y = self.indices[idx]
        group = self.df[self.df['id'] == pid].sort_values('hr')
        
        # 获取特征序列
        X = group[(group['hr'] >= start) & (group['hr'] <= end)][self.feature_cols].values
        
        # 转换为torch张量
        X_tensor = torch.FloatTensor(X)
        y_tensor = torch.FloatTensor(y)

        if self.max_len is not None:
            X_tensor = X_tensor[-self.max_len:]  # 截断到最大长度
            
        seq_len = len(X_tensor)
        
        return X_tensor, y_tensor, seq_len

In [22]:
# 定义特征列
feature_cols = ['gender', 'age', 'height', 'weight',
                'heart_rate', 'sbp', 'mbp', 'dbp', 'resp_rate', 'temperature', 'spo2', 'albumin', 'aniongap', 'bun', 'calcium', 'chloride', 
                'creatinine', 'glucose', 'sodium', 'potassium', 'fibrinogen', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'platelet', 'wbc', 
                'alt', 'ast', 'bilirubin', 'pao2', 'paco2', 'fio2', 'pao2fio2ratio', 'ph', 'baseexcess', 'lactate', 'sao2', 'troponin', 'magnesium', 
                'bnp', 'neutrophils', 'gcs', 'alkaline_phosphatase', 'norepinephrine', 'epinephrine', 'dobutamine', 'dopamine', 'ventilation',
                'lymphocytes', 'bicarbonate', 'urineoutput',
               ]


# feature_cols = ['heart_rate', 'resp_rate', 'spo2', 'temperature', 'sbp', 'mbp', 'urineoutput', 'albumin',
#  'calcium', 'bilirubin', 'bun', 'creatinine', 'glucose', 'bicarbonate', 'hematocrit', 'hemoglobin', 'lactate', 'platelet', 'ptt', 'sodium', 'wbc',
#  'alt', 'ast', 'alkaline_phosphatase', 'inr', 'fio2', 'pao2', 'pao2fio2ratio', 'paco2', 'ph', 'aniongap', 'baseexcess', 'troponin', 'magnesium',
#  'bnp', 'fibrinogen', 'lymphocytes', 'chloride', 'pt', 'neutrophils',
#                ]
    

# 创建数据集和数据加载器
train_dataset = TimeSeriesDataset(train_df, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', stride=4, shuffle=False)
test_dataset = TimeSeriesDataset(test_df, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', stride=4, shuffle=False)

# train_dataset = TimeSeriesDataset(train_df, feature_cols, window_size=24, mode='fix', shuffle=False)
# test_dataset = TimeSeriesDataset(test_df, feature_cols, window_size=24, mode='fix', shuffle=False)

100%|█████████████████████████████████████████████████████████████████████████████| 8210/8210 [00:29<00:00, 274.92it/s]


In [23]:
# 提取标签列
ys = np.array([y for (pid, start, end, y) in train_dataset.indices])
unique, counts = np.unique(ys, return_counts=True)
label_dist = dict(zip(unique, counts))

print("\n使用numpy统计:")
for label, count in label_dist.items():
    print(f"{label}: {count} 个 ({(count/len(ys))*100:.1f}%)")


使用numpy统计:
0: 565868 个 (96.1%)
1: 22996 个 (3.9%)


In [24]:
# 提取标签列
ys = np.array([y for (pid, start, end, y) in test_dataset.indices])
unique, counts = np.unique(ys, return_counts=True)
label_dist = dict(zip(unique, counts))

print("\n使用numpy统计:")
for label, count in label_dist.items():
    print(f"{label}: {count} 个 ({(count/len(ys))*100:.1f}%)")


使用numpy统计:
0: 144733 个 (96.1%)
1: 5880 个 (3.9%)


In [14]:
train_dataset.indices[:10]

[(5, 1, 25, [0]),
 (5, 5, 29, [0]),
 (5, 9, 33, [0]),
 (5, 13, 37, [0]),
 (5, 17, 41, [0]),
 (10, 1, 25, [0]),
 (10, 5, 29, [0]),
 (10, 9, 33, [0]),
 (10, 13, 37, [0]),
 (10, 17, 41, [0])]

In [25]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    自定义collate函数处理变长序列，生成padding mask
    """
    sequences, targets, lengths = zip(*batch)
    # print(targets)
    
    # 按序列长度排序(降序)
    lengths = torch.tensor(lengths)
    lengths, sort_idx = lengths.sort(descending=True)
    sequences = [sequences[i] for i in sort_idx]
    targets = torch.stack([targets[i] for i in sort_idx])
    
    # 填充序列 (batch_first=True)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    
    # 创建padding mask (1表示有效数据，0表示padding)
    batch_size, max_len = sequences_padded.shape[0], sequences_padded.shape[1]
    padding_mask = torch.arange(max_len).expand(batch_size, max_len) < lengths.unsqueeze(1)
    padding_mask = padding_mask.float().unsqueeze(-1)  # (batch_size, max_len, 1)
    
    return sequences_padded, targets, padding_mask

In [26]:
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0, collate_fn=collate_fn)

In [27]:
len(train_dataset)

588864

In [28]:
for x, y, padding_mask in train_loader:
    print(x.shape)
    # print(y)
    # print(padding_mask)
    break

torch.Size([64, 24, 53])


## 检查设备

In [29]:
# Check device 
# Get the GPU device name if available.
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available. {}'.format(torch.cuda.device_count()))
    print('We will use the GPU: {}'.format(torch.cuda.get_device_name(0)))

# If we dont have GPU but a CPU, training will take place on CPU instead
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
torch.cuda.empty_cache()
    
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

There are %d GPU(s) available. 1
We will use the GPU: NVIDIA GeForce RTX 2080


## 定义模型

In [30]:
# 初始化模型
model = itransformerModel(seq_len=24, d_model=128, d_ff=256, e_layers=3, enc_in=len(feature_cols), num_class=2)
# model = lstm_itransformerModel(seq_len=24, d_model=128, d_ff=256, e_layers=3, enc_in=len(feature_cols), num_class=2)
# # 初始化模型
# model = LSTMModel(input_size=len(feature_cols), hidden_size=64, num_layers=2, dropout=0.1, num_class=2)

# model_path = './checkpoint.pth'
# model.load_state_dict(torch.load(model_path))

In [31]:
# model.state_dict()

In [32]:
# torch.load(model_path)

## 训练模型

In [41]:
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import  average_precision_score

# 训练与评估框架
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer,device='cuda'):
        """
        初始化训练器
        参数:
            model: 模型实例
            train_loader: 训练数据加载器
            val_loader: 验证数据加载器
            device: 训练设备
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.best_auc = 0
        self.best_model = None
        
    def train_epoch(self, epoch):
        """训练一个epoch"""
        self.model.train()
        total_loss = 0
        progress_bar = tqdm(self.train_loader, desc=f"Epoch {epoch + 1} [Train]")
        
        for X, y, padding_mask in progress_bar:
            X, y = X.to(self.device), y.to(self.device)
            padding_mask = padding_mask.to(self.device)
            
            # 前向传播
            outputs = self.model(X,padding_mask)
            if y.shape[1] == 1:
                loss = self.criterion(outputs, y.long().squeeze(-1))
                # loss = self.criterion(outputs, y)
            else:
                loss = torch.tensor(0.0, device=self.device)
                for i in range(len(outputs)):
                    loss = loss + self.criterion(outputs[i], y[:, i].long().squeeze(-1))
            # # loss = self.criterion(outputs, y)
            # loss = self.criterion(outputs, y.long().squeeze(-1))
            
            # 反向传播和优化
            self.optimizer.zero_grad()
            loss.backward()
            # nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=4.0)
            self.optimizer.step()
            
            total_loss += loss.item() * X.size(0)
            progress_bar.set_postfix(loss=loss.item())
        
        return total_loss / len(self.train_loader.dataset)
    
    def validate(self, testloader):
        """验证模型"""
        self.model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []

        # total_loss = []
        # preds = []
        # trues = []
        
        with torch.no_grad():
            for X, y, padding_mask in tqdm(testloader, desc="Validating"):
                X, y = X.to(self.device), y.to(self.device)
                padding_mask = padding_mask.to(self.device)
                outputs = self.model(X,padding_mask)
                if y.shape[1] == 1:
                    loss = self.criterion(outputs, y.long().squeeze(-1))
                    # loss = self.criterion(outputs, y)
                else:
                    loss = torch.tensor(0.0, device=self.device)
                    for i in range(len(outputs)):
                        loss = loss + self.criterion(outputs[i], y[:, i].long().squeeze(-1))
                # loss = self.criterion(outputs, y)
                
                val_loss += loss.item() * X.size(0)
                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(y.cpu().numpy())

        
        val_loss /= len(testloader.dataset)
        val_auc = roc_auc_score(all_labels, np.array(all_preds)[:,1])
        val_auprc = average_precision_score(all_labels, np.array(all_preds)[:,1])
        
        # 保存最佳模型
        if val_auc > self.best_auc:
            self.best_auc = val_auc
            self.best_model = self.model.state_dict().copy()
        
        return val_loss, val_auc, val_auprc, np.array(all_preds), all_labels
    
    def train(self, num_epochs=50, early_stop_patience=5):
        """完整训练流程"""
        train_losses = []
        val_losses = []
        val_aucs = []
        
        no_improve = 0
        for epoch in range(num_epochs):
            train_loss = self.train_epoch(epoch)
            val_loss, val_auc, val_auprc = self.validate(self.val_loader)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            val_aucs.append(val_auc)
            
            print(f"Epoch {epoch + 1}/{num_epochs}:")
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            print(f"Val AUC: {val_auc:.4f} | Val AUPRC: {val_auprc:.4f}")

            torch.save(self.model.state_dict(), f'weights_lstmitransformer_cum_24_2_{epoch}epoch.pth')
            # # 早停机制
            # if val_auc > self.best_auc:
            #     no_improve = 0
            # else:
            #     no_improve += 1
            #     if no_improve >= early_stop_patience:
            #         print(f"Early stopping at epoch {epoch + 1}")
            #         break
        
        # 加载最佳模型
        self.model.load_state_dict(self.best_model)
        return train_losses, val_losses, val_aucs

In [42]:
# 训练模型
# criterion = nn.BCELoss()
# weight = torch.tensor([0.025, 0.975]).to(device)
# criterion =  nn.CrossEntropyLoss(weight=weight)
criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# trainer = ModelTrainer(model, train_loader, test_loader, test_loader , criterion, optimizer)
# train_losses, val_losses, val_aucs = trainer.train(num_epochs=5)
# total_loss, accuracy, auroc, auprc = trainer.evaluate(test_loader)

# 训练模型
trainer = ModelTrainer(model, train_loader, test_loader, criterion, optimizer)
# train_losses, val_losses, val_aucs = trainer.train(num_epochs=10)

In [36]:
train_losses, val_losses, val_aucs

NameError: name 'train_losses' is not defined

In [37]:
torch.save(model.state_dict(), 'weights_itransformer_24_24_2_new.pth')

# 外部验证

In [38]:
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, accuracy_score, f1_score, precision_score, average_precision_score
from scipy import stats

def calculate_all_metrics(all_preds, all_labels, threshold=0.5, n_bootstrap=1000, confidence_level=0.95):
    """
    计算所有评估指标，包括Bootstrap置信区间
    
    参数:
    -----------
    all_preds : array-like
        预测概率值
    all_labels : array-like
        真实标签
    threshold : float, default=0.5
        二分类阈值
    n_bootstrap : int, default=1000
        Bootstrap采样次数
    confidence_level : float, default=0.95
        置信水平
    
    返回:
    -----------
    metrics_dict : dict
        包含所有指标及其置信区间的字典
    """
    # 将概率转换为二分类预测
    binary_preds = (all_preds >= threshold).astype(int)
    
    # 计算混淆矩阵
    tn, fp, fn, tp = confusion_matrix(all_labels, binary_preds).ravel()
    
    # 原始指标计算
    metrics = {}
    
    # AUROC
    try:
        metrics['AUROC'] = roc_auc_score(all_labels, all_preds)
    except:
        metrics['AUROC'] = np.nan
    
    # Sensitivity (Recall)
    try:
        metrics['Sensitivity'] = recall_score(all_labels, binary_preds, zero_division=0)
    except:
        metrics['Sensitivity'] = np.nan
    
    # Specificity
    try:
        metrics['Specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0
    except:
        metrics['Specificity'] = np.nan
    
    # Accuracy
    try:
        metrics['Accuracy'] = accuracy_score(all_labels, binary_preds)
    except:
        metrics['Accuracy'] = np.nan
    
    # F1 score
    try:
        metrics['F1_score'] = f1_score(all_labels, binary_preds, zero_division=0)
    except:
        metrics['F1_score'] = np.nan
    
    # Precision
    try:
        metrics['Precision'] = precision_score(all_labels, binary_preds, zero_division=0)
    except:
        metrics['Precision'] = np.nan
    
    # AUPRC
    try:
        metrics['AUPRC'] = average_precision_score(all_labels, all_preds)
    except:
        metrics['AUPRC'] = np.nan
    
    # 额外添加一些可能有用的指标
    metrics['TP'] = tp
    metrics['FP'] = fp
    metrics['TN'] = tn
    metrics['FN'] = fn
    
    # 如果需要Bootstrap，计算置信区间
    if n_bootstrap > 0:
        bootstrap_metrics = _bootstrap_metrics(all_preds, all_labels, threshold, n_bootstrap, confidence_level)
        
        # 合并原始指标和Bootstrap结果
        result_dict = {}
        for key in metrics.keys():
            result_dict[key] = {
                'value': metrics[key],
                'ci_lower': bootstrap_metrics.get(f'{key}_ci_lower', np.nan),
                'ci_upper': bootstrap_metrics.get(f'{key}_ci_upper', np.nan),
                'mean': bootstrap_metrics.get(f'{key}_mean', np.nan),
                'std': bootstrap_metrics.get(f'{key}_std', np.nan)
            }
        
        return result_dict
    
    return metrics

def _bootstrap_metrics(all_preds, all_labels, threshold=0.5, n_bootstrap=1000, confidence_level=0.95):
    """
    Bootstrap采样计算指标置信区间
    
    参数:
    -----------
    all_preds : array-like
        预测概率值
    all_labels : array-like
        真实标签
    threshold : float, default=0.5
        二分类阈值
    n_bootstrap : int, default=1000
        Bootstrap采样次数
    confidence_level : float, default=0.95
        置信水平
    
    返回:
    -----------
    bootstrap_results : dict
        Bootstrap计算的所有指标统计量
    """
    n_samples = len(all_labels)
    indices = np.arange(n_samples)
    
    # 存储所有Bootstrap样本的指标
    bootstrap_auroc = []
    bootstrap_sensitivity = []
    bootstrap_specificity = []
    bootstrap_accuracy = []
    bootstrap_f1 = []
    bootstrap_precision = []
    bootstrap_auprc = []
    
    for i in range(n_bootstrap):
        # 有放回抽样
        bootstrap_indices = np.random.choice(indices, size=n_samples, replace=True)
        
        # 获取Bootstrap样本
        bootstrap_preds = all_preds[bootstrap_indices]
        bootstrap_labels = all_labels[bootstrap_indices]
        
        # 转换为二分类预测
        binary_preds = (bootstrap_preds >= threshold).astype(int)
        
        try:
            # 计算混淆矩阵
            tn, fp, fn, tp = confusion_matrix(bootstrap_labels, binary_preds).ravel()
            
            # 计算各项指标
            # AUROC
            try:
                bootstrap_auroc.append(roc_auc_score(bootstrap_labels, bootstrap_preds))
            except:
                bootstrap_auroc.append(np.nan)
            
            # Sensitivity
            try:
                bootstrap_sensitivity.append(recall_score(bootstrap_labels, binary_preds, zero_division=0))
            except:
                bootstrap_sensitivity.append(np.nan)
            
            # Specificity
            try:
                bootstrap_specificity.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
            except:
                bootstrap_specificity.append(np.nan)
            
            # Accuracy
            try:
                bootstrap_accuracy.append(accuracy_score(bootstrap_labels, binary_preds))
            except:
                bootstrap_accuracy.append(np.nan)
            
            # F1 score
            try:
                bootstrap_f1.append(f1_score(bootstrap_labels, binary_preds, zero_division=0))
            except:
                bootstrap_f1.append(np.nan)
            
            # Precision
            try:
                bootstrap_precision.append(precision_score(bootstrap_labels, binary_preds, zero_division=0))
            except:
                bootstrap_precision.append(np.nan)
            
            # AUPRC
            try:
                bootstrap_auprc.append(average_precision_score(bootstrap_labels, bootstrap_preds))
            except:
                bootstrap_auprc.append(np.nan)
                
        except:
            # 如果Bootstrap样本出现问题，添加NaN值
            bootstrap_auroc.append(np.nan)
            bootstrap_sensitivity.append(np.nan)
            bootstrap_specificity.append(np.nan)
            bootstrap_accuracy.append(np.nan)
            bootstrap_f1.append(np.nan)
            bootstrap_precision.append(np.nan)
            bootstrap_auprc.append(np.nan)
    
    # 计算置信区间
    alpha = 1 - confidence_level
    ci_lower = alpha / 2 * 100
    ci_upper = (1 - alpha / 2) * 100
    
    # 将列表转换为数组，过滤NaN值
    def get_stats(values, metric_name):
        values_array = np.array(values)
        valid_values = values_array[~np.isnan(values_array)]
        
        if len(valid_values) == 0:
            return {
                f'{metric_name}_mean': np.nan,
                f'{metric_name}_std': np.nan,
                f'{metric_name}_ci_lower': np.nan,
                f'{metric_name}_ci_upper': np.nan
            }
        
        mean_val = np.mean(valid_values)
        std_val = np.std(valid_values)
        
        # 使用百分位数法计算置信区间
        ci_lower_val = np.percentile(valid_values, ci_lower)
        ci_upper_val = np.percentile(valid_values, ci_upper)
        
        return {
            f'{metric_name}_mean': mean_val,
            f'{metric_name}_std': std_val,
            f'{metric_name}_ci_lower': ci_lower_val,
            f'{metric_name}_ci_upper': ci_upper_val
        }
    
    # 收集所有指标的Bootstrap结果
    bootstrap_results = {}
    
    # 合并各个指标的统计结果
    for metric_name, values in [
        ('AUROC', bootstrap_auroc),
        ('Sensitivity', bootstrap_sensitivity),
        ('Specificity', bootstrap_specificity),
        ('Accuracy', bootstrap_accuracy),
        ('F1_score', bootstrap_f1),
        ('Precision', bootstrap_precision),
        ('AUPRC', bootstrap_auprc)
    ]:
        bootstrap_results.update(get_stats(values, metric_name))
    
    return bootstrap_results

    
    

In [43]:
external_data_path = 'icu_mortality_' + 'zhejiang' + '.csv'
df_external = pd.read_csv(file_path + external_data_path)
print(df_external.shape)

# df_external = pd.concat([df_external, df_external_1], axis=0)
# print(df_external.shape)

external_dataset = TimeSeriesDataset(df_external, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', stride=4, shuffle=False)
external_loader = DataLoader(external_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)

total_loss, auroc, auprc, all_preds, all_labels = trainer.validate(external_loader)
print("total_loss, auroc, auprc: ", total_loss, auroc, auprc)

(235970, 68)


Validating: 100%|██████████████████████████████████████████████████████████████████| 3230/3230 [00:38<00:00, 84.44it/s]


total_loss, auroc, auprc:  0.11907117988423131 0.8222297972557429 0.2729485432874828


In [51]:
print( "Asia: \n") 
print(calculate_all_metrics(np.array(all_preds)[:, 1].reshape(-1), np.array(all_labels).reshape(-1), threshold=-1.45, n_bootstrap=100))

Asia: 

{'AUROC': {'value': 0.8222297972557429, 'ci_lower': 0.8098901801725736, 'ci_upper': 0.8318340010935624, 'mean': 0.8218920359753694, 'std': 0.006035780951919125}, 'Sensitivity': {'value': 0.74609375, 'ci_lower': 0.7247345106825404, 'ci_upper': 0.7663778204377589, 'mean': 0.7456360858648753, 'std': 0.010572740270272342}, 'Specificity': {'value': 0.739662910142615, 'ci_lower': 0.7364000750868435, 'ci_upper': 0.7437032100885013, 'mean': 0.7398900849467074, 'std': 0.0018676696524967732}, 'Accuracy': {'value': 0.7398540767548528, 'ci_lower': 0.7369477076116197, 'ci_upper': 0.7436318244276287, 'mean': 0.7400607690967854, 'std': 0.0018008286753232981}, 'F1_score': {'value': 0.14567179356806914, 'ci_lower': 0.14030438320112387, 'ci_upper': 0.1523311024720001, 'mean': 0.14607666496736663, 'std': 0.0034035270735293666}, 'Precision': {'value': 0.08071559374559797, 'ci_lower': 0.07739005959181337, 'ci_upper': 0.08461531928946613, 'mean': 0.08097395714868622, 'std': 0.0020372820396734365}, '

In [52]:
total_loss, auroc, auprc, all_preds, all_labels = trainer.validate(test_loader)

Validating: 100%|██████████████████████████████████████████████████████████████████| 2354/2354 [02:22<00:00, 16.55it/s]


In [53]:
print( "Internal test set: \n") 
print(calculate_all_metrics(np.array(all_preds)[:, 1].reshape(-1), np.array(all_labels).reshape(-1), threshold=-1.45, n_bootstrap=100))

Internal test set: 

{'AUROC': {'value': 0.8450462030694005, 'ci_lower': 0.8390794930954066, 'ci_upper': 0.8506218120622188, 'mean': 0.8451303790193903, 'std': 0.0031645735937166804}, 'Sensitivity': {'value': 0.7741496598639456, 'ci_lower': 0.7638290365657506, 'ci_upper': 0.7848504108404685, 'mean': 0.7744291037842155, 'std': 0.005791923122063964}, 'Specificity': {'value': 0.7399418239102347, 'ci_lower': 0.737801835107175, 'ci_upper': 0.7415773166798781, 'mean': 0.7397974489626225, 'std': 0.0010873045558060924}, 'Accuracy': {'value': 0.7412773133793231, 'ci_lower': 0.7390719924574903, 'ci_upper': 0.7430362916879685, 'mean': 0.7411488384136827, 'std': 0.0011092986403854258}, 'F1_score': {'value': 0.18938653242079423, 'ci_lower': 0.18446336530618812, 'ci_upper': 0.19376072481656859, 'mean': 0.1892952040064326, 'std': 0.0023948198195019436}, 'Precision': {'value': 0.10789030835960275, 'ci_lower': 0.10476699433771802, 'ci_upper': 0.11061377587861347, 'mean': 0.10782755519171161, 'std': 0.0

In [56]:
external_data_path = 'icu_mortality_' + 'ams' + '.csv'
df_external_1 = pd.read_csv(file_path + external_data_path)
print(df_external_1.shape)

external_data_path = 'icu_mortality_' + 'salz' + '.csv'
df_external = pd.read_csv(file_path + external_data_path)
print(df_external.shape)

# df_external = pd.concat([df_external, df_external_1], axis=0)
# print(df_external.shape)

# 计算df1的最大id，并顺延df2的id
max_id = df_external_1['id'].max()
df_external['id'] = df_external['id'] + max_id

# 合并DataFrame
df_external = pd.concat([df_external_1, df_external], ignore_index=True)
print(df_external.shape)

external_dataset = TimeSeriesDataset(df_external, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', stride=4, shuffle=False)
external_loader = DataLoader(external_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)

total_loss, auroc, auprc, all_preds, all_labels = trainer.validate(external_loader)
print("total_loss, auroc, auprc: ", total_loss, auroc, auprc)

(341758, 75)
(685979, 75)
(1027737, 75)


Validating: 100%|████████████████████████████████████████████████████████████████| 12719/12719 [04:38<00:00, 45.63it/s]


total_loss, auroc, auprc:  0.11151691214863829 0.806134165340889 0.30206729843716484


In [58]:
print( "Europe: \n") 
print(calculate_all_metrics(np.array(all_preds)[:, 1].reshape(-1), np.array(all_labels).reshape(-1), threshold=-1.5, n_bootstrap=100))

Europe: 

{'AUROC': {'value': 0.806134165340889, 'ci_lower': 0.7982142039695521, 'ci_upper': 0.81247908567915, 'mean': 0.8060668669631604, 'std': 0.0034800816913499537}, 'Sensitivity': {'value': 0.7432337434094903, 'ci_lower': 0.7310962290776154, 'ci_upper': 0.7525579067070758, 'mean': 0.7427227950448837, 'std': 0.005602932552064553}, 'Specificity': {'value': 0.7031348738922283, 'ci_lower': 0.701626733781057, 'ci_upper': 0.7050040886423092, 'mean': 0.7031463740460333, 'std': 0.0008679063078547409}, 'Accuracy': {'value': 0.7042560823992491, 'ci_lower': 0.7027392295709519, 'ci_upper': 0.7061283458724207, 'mean': 0.7042528882489668, 'std': 0.000837579763460565}, 'F1_score': {'value': 0.12322081554755904, 'ci_lower': 0.12002293306200451, 'ci_upper': 0.12583926106121193, 'mean': 0.12314444853414193, 'std': 0.0015659220921480272}, 'Precision': {'value': 0.06717923464281743, 'ci_lower': 0.06533105991929428, 'ci_upper': 0.06870043726367249, 'mean': 0.06713883350123913, 'std': 0.000909440737397

In [59]:
external_data_path = 'icu_mortality_' + 'eicu' + '.csv'
df_external = pd.read_csv(file_path + external_data_path)
print(df_external.shape)

# df_external = pd.concat([df_external, df_external_1], axis=0)
# print(df_external.shape)

external_dataset = TimeSeriesDataset(df_external, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', stride=4, shuffle=False)
external_loader = DataLoader(external_dataset, batch_size=64, shuffle=False, num_workers=0, collate_fn=collate_fn)

total_loss, auroc, auprc, all_preds, all_labels = trainer.validate(external_loader)
print("total_loss, auroc, auprc: ", total_loss, auroc, auprc)

(3084332, 76)


Validating: 100%|██████████████████████████████████████████████████████████████████| 8672/8672 [21:12<00:00,  6.81it/s]


total_loss, auroc, auprc:  0.13613525099675716 0.7825795180021742 0.21246811829090967


In [62]:
print( "US: \n") 
print(calculate_all_metrics(np.array(all_preds)[:, 1].reshape(-1), np.array(all_labels).reshape(-1), threshold=-1.4, n_bootstrap=100))

US: 

{'AUROC': {'value': 0.7825795180021742, 'ci_lower': 0.7791226730539866, 'ci_upper': 0.7861272870439441, 'mean': 0.7826854150559829, 'std': 0.0016939289656860243}, 'Sensitivity': {'value': 0.7280341257868179, 'ci_lower': 0.7222529010220227, 'ci_upper': 0.7349259231936381, 'mean': 0.7281368273644265, 'std': 0.0030468932569052824}, 'Specificity': {'value': 0.6816399213787465, 'ci_lower': 0.6803776531063915, 'ci_upper': 0.6829024958885559, 'mean': 0.6815684790520361, 'std': 0.0006630847033232283}, 'Accuracy': {'value': 0.6832469601193608, 'ci_lower': 0.6820766871607838, 'ci_upper': 0.6844557946936334, 'mean': 0.6831831172201039, 'std': 0.0006432259602059428}, 'F1_score': {'value': 0.13735805352989094, 'ci_lower': 0.13554105971341113, 'ci_upper': 0.13931725231371622, 'mean': 0.13746402970755647, 'std': 0.0010757086043705292}, 'Precision': {'value': 0.07583269665295772, 'ci_lower': 0.07472047384706108, 'ci_upper': 0.07700789418861624, 'mean': 0.07589658883710886, 'std': 0.0006398350222